In [2]:
import time
import os
import tensorflow as tf

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
    
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

GPU found


In [3]:
data = pd.read_csv('emnist-byclass-mapping.txt', sep=" ", header=None)
data.columns = ['Target','ASCII']

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [4]:
aug_df = pd.read_csv('augmented_emnist.csv')
aug_df.rename(columns={'25':'Target'}, inplace=True)

# train_df=pd.merge(train_df, data, how='left', on='Target')
# train_df=train_df.drop(['Target','ASCII'],axis=1)
# train_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

train_y = np.array(aug_df.pop('Target'))
train_x = aug_df

train_x = train_x / 255.0
train_x = train_x.values.reshape([-1, 28, 28, 1])
train_y = keras.utils.to_categorical(train_y)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [5]:
test_df = pd.read_csv('emnist-byclass-test.csv')
test_df.rename(columns={'18':'Target'}, inplace=True)

# test_df=pd.merge(test_df, data, how='left', on='Target')
# test_df=test_df.drop(['Target','ASCII'],axis=1)
# test_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

test_y = np.array(test_df.pop('Target'))
test_x = test_df

test_x = test_x / 255.0
test_x = test_x.values.reshape([-1, 28, 28, 1])
test_y = keras.utils.to_categorical(test_y)

In [6]:
hand_df = pd.read_csv('our_handmade_dataset.csv')
hand_df.rename(columns={'0':'Target'}, inplace=True)

# hand_df=pd.merge(hand_df, data, how='left', on='Target')
# hand_df=hand_df.drop(['Target','ASCII'],axis=1)
# hand_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

hand_y = np.array(hand_df.pop('Target'))
hand_x = hand_df

hand_x = hand_x / 255.0
hand_x = hand_x.values.reshape([-1, 28, 28, 1])
hand_y = keras.utils.to_categorical(hand_y)

In [7]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode="constant",
    cval=0
)

In [7]:
#new VGG
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = BatchNormalization()(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

new_vgg_emnist_model = Model (inputs=inputs, outputs =output)
new_vgg_emnist_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        36928     
                                                             

In [11]:
new_vgg_emnist_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_new_model.h5", save_best_only=True)
  
new_vgg_emnist_model.fit(train_x, train_y, batch_size = 64, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

new_vgg_emnist_model.evaluate(test_x, test_y)

Epoch 1/100
6915/6915 [==============================] - 123s 17ms/step - loss: 2.1919 - accuracy: 0.3485 - val_loss: 208973119488.0000 - val_accuracy: 0.5454
Epoch 2/100
6915/6915 [==============================] - 128s 18ms/step - loss: 0.9336 - accuracy: 0.6361 - val_loss: 1.0650 - val_accuracy: 0.6841
Epoch 3/100
6915/6915 [==============================] - 128s 19ms/step - loss: 0.7390 - accuracy: 0.6956 - val_loss: 3293128687616.0000 - val_accuracy: 0.6661
Epoch 4/100
6915/6915 [==============================] - 128s 18ms/step - loss: 0.6671 - accuracy: 0.7206 - val_loss: 2.3272 - val_accuracy: 0.7295
Epoch 5/100
6915/6915 [==============================] - 128s 18ms/step - loss: 0.6240 - accuracy: 0.7346 - val_loss: 0.7051 - val_accuracy: 0.7368
Epoch 6/100
6915/6915 [==============================] - 127s 18ms/step - loss: 0.5945 - accuracy: 0.7442 - val_loss: 1.3769 - val_accuracy: 0.7420
Epoch 7/100
6915/6915 [==============================] - 126s 18ms/step - loss: 0.5721 - 

KeyboardInterrupt: 

In [14]:
#new VGG+Dropout
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 4096, activation ='relu')(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

nnnew_vgg_emnist_model = Model (inputs=inputs, outputs =output)
nnnew_vgg_emnist_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_39 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_40 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 14, 14, 64)        18496     
                                                                 
 conv2d_42 (Conv2D)          (None, 14, 14, 64)        36928     
                                                           

In [15]:
nnnew_vgg_emnist_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_new_model.h5", save_best_only=True)
  
nnnew_vgg_emnist_model.fit(train_x, train_y, batch_size = 64, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

nnnew_vgg_emnist_model.evaluate(test_x, test_y)

Epoch 1/100
6915/6915 [==============================] - 117s 17ms/step - loss: 1.6512 - accuracy: 0.4536 - val_loss: 0.8766 - val_accuracy: 0.6538
Epoch 2/100
6915/6915 [==============================] - 116s 17ms/step - loss: 0.7551 - accuracy: 0.6878 - val_loss: 0.7833 - val_accuracy: 0.6914
Epoch 3/100
6915/6915 [==============================] - 117s 17ms/step - loss: 0.6746 - accuracy: 0.7189 - val_loss: 4.1317 - val_accuracy: 0.7079
Epoch 4/100
6915/6915 [==============================] - 117s 17ms/step - loss: 0.6349 - accuracy: 0.7313 - val_loss: 0.6226 - val_accuracy: 0.7409
Epoch 5/100
6915/6915 [==============================] - 115s 17ms/step - loss: 0.6112 - accuracy: 0.7393 - val_loss: 0.8954 - val_accuracy: 0.7271
Epoch 6/100
6915/6915 [==============================] - 112s 16ms/step - loss: 0.5930 - accuracy: 0.7463 - val_loss: 0.6197 - val_accuracy: 0.7437
Epoch 7/100
6915/6915 [==============================] - 115s 17ms/step - loss: 0.5770 - accuracy: 0.7507 - val_

[0.6289998292922974, 0.7151011824607849]

In [16]:
#new VGG+Dropout
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 4096, activation ='relu')(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

vgg_emnist_model = Model (inputs=inputs, outputs =output)
vgg_emnist_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_52 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_53 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_4 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 14, 14, 64)        1849

In [17]:
vgg_emnist_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_new_model.h5", save_best_only=True)
  
vgg_emnist_model.fit(train_x, train_y, batch_size = 64, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

vgg_emnist_model.evaluate(test_x, test_y)

Epoch 1/100
6915/6915 [==============================] - 121s 17ms/step - loss: 1.5103 - accuracy: 0.4852 - val_loss: 0.9770 - val_accuracy: 0.6364
Epoch 2/100
6915/6915 [==============================] - 119s 17ms/step - loss: 0.7621 - accuracy: 0.6834 - val_loss: 0.8163 - val_accuracy: 0.6833
Epoch 3/100
6915/6915 [==============================] - 120s 17ms/step - loss: 0.6859 - accuracy: 0.7128 - val_loss: 0.6988 - val_accuracy: 0.7091
Epoch 4/100
6915/6915 [==============================] - 122s 18ms/step - loss: 0.6473 - accuracy: 0.7272 - val_loss: 0.7001 - val_accuracy: 0.7189
Epoch 5/100
6915/6915 [==============================] - 123s 18ms/step - loss: 0.6217 - accuracy: 0.7352 - val_loss: 0.6777 - val_accuracy: 0.7182
Epoch 6/100
6915/6915 [==============================] - 123s 18ms/step - loss: 0.6038 - accuracy: 0.7419 - val_loss: 0.6192 - val_accuracy: 0.7381
Epoch 7/100
6915/6915 [==============================] - 123s 18ms/step - loss: 0.5896 - accuracy: 0.7455 - val_

[0.5725254416465759, 0.7600539922714233]

In [18]:
#new VGG+Dropout
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 4096, activation ='relu')(x)
x = BatchNormalization()(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

nnew_vgg_emnist_model = Model (inputs=inputs, outputs =output)
nnew_vgg_emnist_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_65 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_66 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_10 (Bat  (None, 28, 28, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_67 (Conv2D)          (None, 14, 14, 64)        1849

In [ ]:
nnew_vgg_emnist_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_new_model.h5", save_best_only=True)
  
nnew_vgg_emnist_model.fit(train_x, train_y, batch_size = 64, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

nnew_vgg_emnist_model.evaluate(test_x, test_y)

Epoch 1/100
6915/6915 [==============================] - 132s 19ms/step - loss: 1.6640 - accuracy: 0.4856 - val_loss: 3.9450 - val_accuracy: 0.6169
Epoch 2/100
6915/6915 [==============================] - 128s 19ms/step - loss: 0.7625 - accuracy: 0.6885 - val_loss: 0.7550 - val_accuracy: 0.6998
Epoch 3/100
6915/6915 [==============================] - 128s 19ms/step - loss: 0.6850 - accuracy: 0.7148 - val_loss: 0.6758 - val_accuracy: 0.7188
Epoch 4/100
6915/6915 [==============================] - 129s 19ms/step - loss: 0.6467 - accuracy: 0.7265 - val_loss: 0.9056 - val_accuracy: 0.7293
Epoch 5/100
1434/6915 [=====>........................] - ETA: 1:34 - loss: 0.6142 - accuracy: 0.7342

In [20]:
#orginal VGG
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x) 
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = BatchNormalization()(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

vgg_emnist_model = Model (inputs=inputs, outputs =output)
vgg_emnist_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_59 (Conv2D)          (None, 28, 28, 64)        640       
                                                                 
 conv2d_60 (Conv2D)          (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_61 (Conv2D)          (None, 14, 14, 128)       73856     
                                                                 
 conv2d_62 (Conv2D)          (None, 14, 14, 128)       147584    
                                                           

In [21]:
vgg_emnist_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_new_model.h5", save_best_only=True)

vgg_emnist_model.fit(train_x, train_y, batch_size = 64, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

vgg_emnist_model.evaluate(test_x, test_y)

Epoch 1/100
6915/6915 [==============================] - 208s 30ms/step - loss: 3.9742 - accuracy: 0.0692 - val_loss: nan - val_accuracy: 0.0152
Epoch 2/100
 447/6915 [>.............................] - ETA: 3:00 - loss: 4.1877 - accuracy: 0.0161

KeyboardInterrupt: 